# Key Terminology [](url)
## Munging

In [0]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

In [0]:
df1=spark.read.csv("/Volumes/lakehouse1/dbread/read_volume/commondata/custsmodified",header=False,inferSchema=True).toDF("id","name","lname","age","prof")
display(df1)
df1.printSchema()
display(df1.columns)
display(df1.schema)
